In [1]:
!pip install h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.0 MB/s eta 0:00:00


In [3]:
# --------------------------------------------
# 0️⃣ Librerías necesarias
# --------------------------------------------
import pandas as pd
import h5py


In [4]:
# --------------------------------------------
# 2️⃣ Cargar dataset parquet
# --------------------------------------------

# Ruta local (ajusta si hace falta)
file_path = "panel_cliente_producto_fe.parquet"

df = pd.read_parquet(file_path)

df['fecha'] = pd.to_datetime(df['fecha'])

print(f"✅ Dataset cargado con forma: {df.shape}")
df.head()


✅ Dataset cargado con forma: (12138186, 194)


,customer_id,product_id,periodo,tn,IPC,inflacion,cambio_dolar,dias_feriados,cat1,cat2,...,cluster_dtw_factorized,cluster_x_month,periodo_factorized,cat1_factorized,cat2_factorized,cat3_factorized,brand_factorized,descripcion_factorized,product_target_enc,customer_target_enc
0,10001,20001,201701,99.43861,102,2.000000,16.080,1,HC,ROPA LAVADO,...,0,0,0,0,0,0,0,0,2.882842,4.146726
1,10001,20001,201702,198.84365,104,1.960784,15.800,2,HC,ROPA LAVADO,...,0,0,1,0,0,0,0,0,2.882842,4.146726
2,10001,20001,201703,92.46537,107,2.884615,15.645,1,HC,ROPA LAVADO,...,0,0,2,0,0,0,0,0,2.882842,4.146726
3,10001,20001,201704,13.29728,109,1.869159,15.490,1,HC,ROPA LAVADO,...,0,0,3,0,0,0,0,0,2.882842,4.146726
4,10001,20001,201705,101.00563,111,1.834862,16.185,2,HC,ROPA LAVADO,...,0,0,4,0,0,0,0,0,2.882842,4.146726


In [5]:
import pandas as pd

# Asegurar datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Fechas objetivo
fechas_target = pd.to_datetime([
    "2019-09-01",
    "2019-10-01",
    "2019-11-01"
])

# Filtrar fechas específicas
df_sub = df[df['fecha'].isin(fechas_target)]

# Agrupar por customer-product y contar presencia
series_validas = (
    df_sub.groupby(['customer_id', 'product_id'])['fecha']
    .nunique()
    .reset_index()
    .query("fecha == 3")  # Solo los que tienen las 3 fechas
)

# Hacer merge para quedarnos solo con esas series
df_clust = df_sub.merge(series_validas[['customer_id', 'product_id']], on=['customer_id', 'product_id'])


In [6]:
# Pivot para tener una fila por serie y una columna por mes
df_wide = df_clust.pivot_table(
    index=['customer_id', 'product_id'],
    columns='fecha',
    values='tn'
)

# Convertir a numpy array para clustering
X = df_wide.values




In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

# Convertir a formato compatible con tslearn
X_ts = to_time_series_dataset(X)

# Elegir número de clusters (puedes afinar luego)
n_clusters = 50

# Modelo con DTW como métrica
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=10, random_state=42)
labels = model.fit_predict(X_ts)



In [ ]:
# Añadir etiquetas
df_clusters = df_wide.copy()
df_clusters['cluster'] = labels

# Recuperar customer_id y product_id
df_clusters.reset_index(inplace=True)


In [ ]:
# Guardar resultado
df_clusters.to_csv("clustering_dtw_3meses.csv", index=False)

print(f"✅ Clustering terminado. Total series: {df_clusters.shape[0]}")
df_clusters.head()
